# Product Amount Forecast by Time

## Data

Info about this data set: https://fred.stlouisfed.org/series/IPN31152N


Units:  Index 2012=100, Not Seasonally Adjusted

Frequency:  Monthly

The industrial production (IP) index measures the real output of all relevant establishments located in the United States, regardless of their ownership, but not those located in U.S. territories.

NAICS = 31152

Source Code: IP.N31152.N

Suggested Citation:
Board of Governors of the Federal Reserve System (US), Industrial Production: Nondurable Goods: Ice cream and frozen dessert [IPN31152N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/IPN31152N, November 16, 2019.

In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.rcParams["figure.figsize"] = (10,6)

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set it None to display all rows in the dataframe
# pd.set_option('display.max_rows', None)

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)

In [182]:
df = pd.read_csv('../input/frozen-dessert-production/Frozen_Dessert_Production.csv',
                 index_col='DATE',
                 parse_dates=True
                )
df

## Exploratory Data Analysis and Visualization

1. Implement basic steps to see how is your data looks like
2. Change the column name as "Production"
2. Plot your data and see its seasonalty

In [183]:
df.info()

In [184]:
df.describe()

In [185]:
df.rename(columns={'IPN31152N':'Production'}, inplace=True)
df

In [186]:
df.plot()

## Preprocessing of Data

### Train Test Split

In [187]:
len(df)

In [188]:
test_size = 24

In [189]:
test_ind = len(df)- test_size
test_ind

In [190]:
train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

In [191]:
train

In [192]:
test

### Scaling

In [193]:
from sklearn.preprocessing import MinMaxScaler


In [194]:
scaler = MinMaxScaler()


In [195]:
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

## Time Series Generator

In [196]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [197]:
length = 12
batch_size = 1
generator = TimeseriesGenerator(train_scaled, train_scaled, length = length, batch_size = batch_size)

In [198]:
len(generator)

In [199]:
generator[0]

In [200]:
X, y = generator[0]

In [201]:
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

## Modelling & Model Performance

### Import related libraries

In [202]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

### Creating Model

In [203]:
n_features = train_scaled.shape[1]

## With "tanh" activation function

In [204]:
model = Sequential()
model.add(LSTM(100, activation = 'tanh', return_sequences=True, input_shape = (length, n_features)))
model.add(LSTM(50, activation = 'tanh'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

In [205]:
model.fit_generator(generator = generator, epochs = 5)

In [206]:
loss_df = pd.DataFrame(model.history.history)
loss_df.plot();

#### Evaluate on Test Data

In [207]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [208]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    score = r2_score(actual, pred)
    return print("r2_score:", score, "\nmae:", mae, "\nmse:",mse, "\nrmse:",rmse)

In [209]:
test

In [210]:
predictions_scaled = []

first_eval_batch = train_scaled[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead
    current_pred = model.predict(current_batch)
    
    # store prediction
    predictions_scaled.append(current_pred[0]) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:, 1:, :], [current_pred], axis = 1)

In [211]:
predictions_scaled

In [212]:
test_scaled

#### Inverse Transformations and Comparing

In [213]:
predictions = scaler.inverse_transform(predictions_scaled)

In [214]:
predictions

In [215]:
test

In [216]:
test['RNN_tanh'] = predictions

In [217]:
test.plot();

#### Calculation R2 Score and Error Metrics

In [218]:
eval_metrics(test.Production, test.RNN_tanh)

## With "relu" activation function

In [219]:
model = Sequential()
model.add(LSTM(100, activation = 'relu', return_sequences=True, input_shape = (length, n_features)))
model.add(LSTM(50, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

In [220]:
model.summary()

In [221]:
model.fit_generator(generator = generator, epochs = 5)

In [222]:
loss_df = pd.DataFrame(model.history.history)
loss_df.plot();

#### Evaluate on Test Data

In [223]:
predictions_scaled = []

first_eval_batch = train_scaled[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead
    current_pred = model.predict(current_batch)
    
    # store prediction
    predictions_scaled.append(current_pred[0]) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:, 1:, :], [current_pred], axis = 1)

In [224]:
predictions_scaled

In [225]:
test_scaled

#### Inverse Transformations and Comparing

In [226]:
predictions = scaler.inverse_transform(predictions_scaled)

In [227]:
predictions

In [228]:
test

In [229]:
test['RNN_relu'] = predictions

In [230]:
test.plot();

#### Calculation R2 Score and Error Metrics

In [231]:
eval_metrics(test.Production, test.RNN_relu)

## Retrain and Forecasting

Select activation function providing a better score, create your final model with full data, forecast for the next 12 months, and plot this forecast.

In [232]:
full_scaler = MinMaxScaler()
scaled_full_data = full_scaler.fit_transform(df)

In [233]:
generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length = length, batch_size = batch_size)

In [234]:
len(df)

In [235]:
len(generator)

In [250]:
model = Sequential()
model.add(LSTM(100, activation = 'relu', return_sequences=True, input_shape = (length, n_features)))
model.add(LSTM(50, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

model.fit_generator(generator, epochs=15)

In [251]:
scaled_full_data.shape

In [252]:
scaled_full_data[-length:].shape

In [253]:
forecast = []
# Replace periods with whatever forecast length you want

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(length):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)
    
    # store prediction
    forecast.append(current_pred[0]) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:, 1:, :], [current_pred], axis = 1)

In [254]:
forecast = scaler.inverse_transform(forecast)

In [255]:
forecast

In [256]:
df

In [257]:
forecast_index = pd.date_range(start = '2019-10-01', periods = length, freq = 'MS')

In [258]:
forecast_index

In [259]:
forecast_df = pd.DataFrame(data = forecast, index = forecast_index, columns = ['Forecast'])

In [260]:
forecast_df

In [261]:
plt.figure(figsize = (16, 8))
plt.plot(df.index, df['Production'])
plt.plot(forecast_df.index, forecast_df['Forecast'])

### Joining pandas plots

In [248]:
ax = df.plot()
forecast_df.plot(ax = ax, figsize = (16, 8));

In [249]:
ax = df.plot()
forecast_df.plot(ax = ax, figsize = (16, 8))
plt.xlim('2018-01-01', '2020-12-01')